# Shunt blocks

In PowSyBl, the shunt compensators consist of discrete blocks which can be defined either with a linear model or with a non-linear (discrete) model. This particularity requires a more detailed initialization of this element, which is explained in this notebook.

## Linear shunts

The blocks of linear shunts are identical between each other, meaning that we only need the size of one of them and the total amount in order to model completely the element. To create this shunt we need a linear_shunt_df, and then define the element as follows:


In [3]:
import pypowsybl as pp
import pandas as pd
import numpy as np

grid = pp.network.create_empty()
grid.create_substations(id=['ss1'])
grid.create_voltage_levels(id=['vl1'], substation_id=['ss1'], topology_kind=['BUS_BREAKER'], nominal_v=[10] )
grid.create_buses(id=['b1'], voltage_level_id=['vl1'])


linear_model_df = pd.DataFrame.from_records(
    index='id',
    columns=['id', 'g_per_section', 'b_per_section', 'max_section_count'],
    data=[('LSH', 0.1, -0.01, 5)])

linear_shunt_df = pd.DataFrame.from_records(
    index='id',
    columns=['id', 'name', 'model_type', 'section_count', 'target_v',
             'target_deadband', 'voltage_level_id', 'bus_id'],
    data=[('LSH', 'linear_shunt', 'LINEAR', 5, 10, 2, 'vl1', 'b1')])

grid.create_shunt_compensators(linear_shunt_df, linear_model_df)


In [4]:
grid.get_shunt_compensators(all_attributes=True)

,name,g,b,model_type,max_section_count,section_count,solved_section_count,voltage_regulation_on,target_v,target_deadband,regulating_bus_id,p,q,i,voltage_level_id,bus_id,bus_breaker_bus_id,node,connected,fictitious
id,,,,,,,,,,,,,,,,,,,,
LSH,linear_shunt,0.5,-0.05,LINEAR,5,5,NaN,False,10.0,2.0,vl1_0,NaN,NaN,NaN,vl1,vl1_0,b1,-1,True,False


In [5]:
grid.update_shunt_compensators(id='LSH', section_count=3)
grid.get_shunt_compensators(all_attributes=True)

,name,g,b,model_type,max_section_count,section_count,solved_section_count,voltage_regulation_on,target_v,target_deadband,regulating_bus_id,p,q,i,voltage_level_id,bus_id,bus_breaker_bus_id,node,connected,fictitious
id,,,,,,,,,,,,,,,,,,,,
LSH,linear_shunt,0.3,-0.03,LINEAR,5,3,NaN,False,10.0,2.0,vl1_0,NaN,NaN,NaN,vl1,vl1_0,b1,-1,True,False


But if we try to set a section_count higher than the total amount, we will get an error saying that the section count is outside the limit.

In [6]:
grid.update_shunt_compensators(id='LSH', section_count=6)
grid.get_shunt_compensators(all_attributes=True)

PyPowsyblError: Shunt compensator 'LSH': the current number (6) of section should be lesser than the maximum number of section (5)

## Non-linear shunts

For these types of shunts, the initialitation now requires the input of the individual blocks instead

In [7]:
nonlinear_model_df = pd.DataFrame.from_records(
    index='id',
    columns=['id', 'g', 'b'],
    data=[('NLSH', 0.1, -0.01),
          ('NLSH', 0.3, -0.03),
          ('NLSH', 0.5, -0.05)])

nonlinear_shunt_df = pd.DataFrame.from_records(
    index='id',
    columns=['id', 'name', 'model_type', 'section_count', 'target_v',
             'target_deadband', 'voltage_level_id', 'bus_id'],
    data=[('NLSH', 'non_linear_shunt', 'NON_LINEAR', 3, 10, 2, 'vl1', 'b1')])

grid.create_shunt_compensators(shunt_df=nonlinear_shunt_df, non_linear_model_df=nonlinear_model_df)
grid.get_shunt_compensators(all_attributes=True)

,name,g,b,model_type,max_section_count,section_count,solved_section_count,voltage_regulation_on,target_v,target_deadband,regulating_bus_id,p,q,i,voltage_level_id,bus_id,bus_breaker_bus_id,node,connected,fictitious
id,,,,,,,,,,,,,,,,,,,,
LSH,linear_shunt,0.3,-0.03,LINEAR,5,3,NaN,False,10.0,2.0,vl1_0,NaN,NaN,NaN,vl1,vl1_0,b1,-1,True,False
NLSH,non_linear_shunt,0.5,-0.05,NON_LINEAR,3,3,NaN,False,10.0,2.0,vl1_0,NaN,NaN,NaN,vl1,vl1_0,b1,-1,True,False
